<a href="https://colab.research.google.com/github/Esbern/Python-for-Planners/blob/main/project_example/rain_surfice_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simplified Water Flow Model: Description and Assumptions**

## **1. Overview of the Model**
This simplified water flow model simulates surface water movement over a digital terrain model (DTM) using a grid-based approach. The model:

- Represents terrain using a **hat-shaped DTM**, where elevation is highest in the center and decreases outward.
- Uses a **raster-based approach**, where each cell has an elevation and water depth value.
- Simulates **rainfall**, **infiltration**, and **water flow** between cells over multiple timesteps.
- Implements **basic flow dynamics**, where water moves from higher to lower elevation cells.

## **2. Assumptions and Limitations**
While this model provides a conceptual understanding of surface water movement, it includes several simplifications:

1. **Flow Direction and Water Distribution:**
   - Water only flows to the **four nearest neighboring cells** (north, south, east, west), which ignores diagonal movement.
   - The flow amount is proportional to **elevation difference**, but it does not consider inertia, turbulence, or conservation of energy principles from the Shallow Water Equations (SWE).

2. **Rainfall and Infiltration:**
   - Rainfall is applied **uniformly** across the entire grid, which does not account for variations due to storms or wind effects.
   - Infiltration is constant across the landscape, assuming **homogeneous soil properties**, which is unrealistic in real-world conditions.
   
3. **No Evaporation or Water Retention:**
   - The model does not account for **evaporation** or **water retention** in depressions.
   - Water in low-lying areas may not be accurately trapped, leading to unrealistic continuous flow instead of pooling.

4. **No Subsurface Flow or Groundwater Interaction:**
   - The model does not simulate **water percolation** into deeper soil layers or interactions with groundwater.

5. **No Explicit Time-Stepping Calculation:**
   - The model assumes a fixed timestep for each iteration but does not dynamically adjust based on flow velocity or stability constraints.

---

## **3. Recommendations for Improving the Model**
Students can improve the realism of the model by implementing the following enhancements:

### **A. More Realistic Water Flow Dynamics**
- **Use multiple flow directions (D8 method)**: Allow water to flow diagonally to adjacent cells to improve accuracy.
- **Implement a physically based flow formula**: Instead of a simple proportional rule, consider using:
  - **Manning's equation** to model flow resistance.
  - **Shallow Water Equations (SWE)** to capture momentum and wave effects.

### **B. More Advanced Rainfall and Infiltration Handling**
- **Vary rainfall intensity over time**: Introduce different rainfall rates across timesteps to simulate storms.
- **Use spatially variable infiltration rates**: Assign different infiltration capacities based on soil type.
- **Implement saturation effects**: Modify infiltration rates based on current soil water content.

### **C. Hydrological Enhancements**
- **Add evaporation and retention**: Introduce evaporation and allow water to collect in depressions.
- **Include subsurface flow**: Model water infiltration into deeper soil layers to simulate groundwater movement.
- **Simulate rivers and drainage systems**: Introduce structured water pathways where flow concentrates.

### **D. Computational Improvements**
- **Use a finer resolution grid**: Increase spatial resolution for more detailed terrain interactions.
- **Optimize time-stepping**: Adjust timestep dynamically based on flow velocity to ensure stability.
- **Parallelize computations**: Utilize libraries like **Numba** or **CuPy** for faster processing.

---

## **4. Conclusion**
This simple model provides a **basic but effective way to teach water flow dynamics** in a controlled environment. While it lacks real-world hydrodynamic complexity, students can build upon it by implementing the recommended improvements to create a more **realistic and physically accurate** water simulation.

By progressively adding complexity, students can explore how real-world hydrological models are developed and used for environmental analysis, flood prediction, and urban drainage planning.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# GRID INITIALIZATION
# -------------------------------

# Define grid size (number of cells in x and y directions)
nx, ny = 50, 50  # 50x50 grid

# Create a "hat-shaped" Digital Terrain Model (DTM)
# This creates a peak in the center and a gradual slope downward
x = np.linspace(-1, 1, nx)
y = np.linspace(-1, 1, ny)
X, Y = np.meshgrid(x, y)
dtm = 1 - (X**2 + Y**2)  # Parabolic hill shape

# Store the original DTM for visualization
original_dtm = np.copy(dtm)

# Initialize a grid for water depth (initially zero everywhere)
water_depth = np.zeros((nx, ny))

# -------------------------------
# INPUT PARAMETERS
# -------------------------------

# Rainfall per timestep (uniform rain over the grid)
rainfall = np.full((nx, ny), 0.01)  # 1 cm of rain per timestep

# Infiltration per timestep (constant for all cells)
infiltration = np.full((nx, ny), 0.002)  # 2 mm infiltration per timestep

# Number of timesteps for simulation
timesteps = 100

# -------------------------------
# NEIGHBOR IDENTIFICATION FUNCTION
# -------------------------------

def get_neighbors(i, j):
    """
    Returns the neighboring cell indices (north, south, west, east) if within bounds.
    """
    neighbors = []
    for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # N, S, W, E
        ni, nj = i + di, j + dj
        if 0 <= ni < nx and 0 <= nj < ny:
            neighbors.append((ni, nj))
    return neighbors

# -------------------------------
# VISUALIZE ORIGINAL DTM
# -------------------------------
plt.imshow(original_dtm, cmap="terrain", origin="lower")
plt.colorbar(label="Elevation")
plt.title("Original DTM (Terrain Model)")
plt.show()

# -------------------------------
# SIMULATION LOOP
# -------------------------------

# Iterate over multiple timesteps to model water accumulation and flow
for t in range(timesteps):
    # Create a new water depth grid for the next timestep
    new_water_depth = np.copy(water_depth)

    # Step 1: Add Rainfall to each cell
    new_water_depth += rainfall

    # Step 2: Apply Infiltration (cannot infiltrate more water than available in the cell)
    new_water_depth -= np.minimum(new_water_depth, infiltration)

    # Step 3: Compute Flow Between Neighboring Cells
    for i in range(nx):
        for j in range(ny):
            # Calculate the total height (terrain + water depth)
            cell_elevation = dtm[i, j] + water_depth[i, j]
            neighbors = get_neighbors(i, j)

            # Identify possible outflow to lower neighbors
            total_flow = 0
            flow_out = []
            for ni, nj in neighbors:
                neighbor_elevation = dtm[ni, nj] + water_depth[ni, nj]
                if cell_elevation > neighbor_elevation:  # Only flow downhill
                    flow_amount = (cell_elevation - neighbor_elevation) * 0.1  # Simplified flow formula
                    total_flow += flow_amount
                    flow_out.append((ni, nj, flow_amount))

            # Distribute Flow Proportionally
            if total_flow > 0:
                for ni, nj, flow_amount in flow_out:
                    transfer_amount = (flow_amount / total_flow) * water_depth[i, j]  # Ensure conservation of mass
                    new_water_depth[i, j] -= transfer_amount
                    new_water_depth[ni, nj] += transfer_amount

    # Update water depth for the next timestep
    water_depth = new_water_depth

    # -------------------------------
    # VISUALIZATION
    # -------------------------------
    if t % 10 == 0:  # Update visualization every 10 timesteps
        plt.imshow(water_depth, cmap="Blues", origin="lower")
        plt.colorbar(label="Water Depth (m)")
        plt.title(f"Water Depth at Time Step {t}")
        plt.show()

# Final visualization
plt.imshow(water_depth, cmap="Blues", origin="lower")
plt.colorbar(label="Water Depth (m)")
plt.title("Final Water Depth Distribution")
plt.show()
